# Data Cleaning and Exploration: Transforming a Messy FIFA 21 Dataset

In this data cleaning project, I'll be putting my Python skills to the test by tackling a messy dataset sourced from Kaggle. The dataset, available [here](https://www.kaggle.com/datasets/yagunnersya/fifa-21-messy-raw-dataset-for-cleaning-exploring?select=fifa21_raw_data.csv%E2%80%8B), presents a prime opportunity to apply the techniques I've learned to clean and explore real-world data. Through this project, I aim to demonstrate my ability to handle data quality issues, such as missing values and inconsistencies. Follow along as I navigate the process of transforming raw, disorderly data into a structured and informative dataset ready for analysis and interpretation.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import re

In [2]:
filepath = r'C:\Users\dlam1\OneDrive\Desktop\Data Analysis Portfolio\FIFA 21 Messy Dataset\fifa21 raw data v2.csv'

fifa = pd.read_csv(filepath)
fifa.head()

## Removing Unneeded Columns

We are eliminating the 'photoURL' and 'playerURL' columns from our dataset as they do not contribute to our analysis.

In [3]:
fifa = fifa.drop(columns=['photoUrl', 'playerUrl'])

## Renaming Column Name

I standardized the column name from '↓OVA' to 'OVA' to adhere to consistent naming conventions.

In [4]:
fifa = fifa.rename(columns={'↓OVA': 'OVA'})

## Finding Duplicated Rows and Null Values

In [5]:
print(f"There are {fifa.duplicated().sum()} duplicated rows.")

In [6]:
fifa.info()

In the dataset, we have observed null values in the 'Loan Date End' and 'Hits' columns.

In [7]:
# these are the cols which have null values
fifa.columns[fifa.isnull().sum() != 0]

For the 'Loan Date End' column, null values indicate when a player is on loan and not under contract, so we won't address them here. However, we will handle the null values in the 'Hits' column by first calculating the percentage of nulls.

In [8]:
hit_null_percentages = fifa['Hits'].isnull().value_counts() / fifa.shape[0]
null_percentage = hit_null_percentages[True]
print(f"The hits column has a null percentage of {null_percentage:.2%}.")

Given the presence of missing values in the Hits column, we'll fill these null values with 0. This choice signifies a deliberate representation of the lack of information, indicating that the data was not available rather than unintentionally missing.

In [9]:
fifa['Hits'].fillna(0, inplace=True)

## Column Specific Cleaning

### Club

In the 'Club' column, there are leading newline characters present in each entry. To clean this, we will remove these leading newline characters.

In [10]:
fifa["Club"].unique()

In [11]:
# remove newlines from club col
fifa['Club'] = fifa['Club'].replace(r"\n","",regex=True)
fifa['Club']

### Contract

The 'Contract' column mostly contains year ranges indicating when each player signed their contract.

In [12]:
fifa["Contract"].unique()

In the 'Contract' column, besides the year ranges, we also observe entries indicating when the contract is either on loan or free.

In [13]:
# entries of contract col where it has no ~ delimiter
fifa[~fifa["Contract"].str.contains('~')]["Contract"].unique()

Initially, we will split the 'Contract' column into two separate columns: 'Beginning Year' and 'Ending Year'. This will represent the start and end years of each FIFA player's contract.

In [14]:
# break contract col into beginning year and ending year cols
fifa[['Beginning Year', 'Ending Year']] = fifa['Contract'].str.split(' ~ ', expand=True)
fifa

In the 'Contract' column, some entries are labeled as 'On Loan' or 'Free', while others contain year ranges with a '~' delimiter. To address this, we will use the 'Loan Date End' column for entries labeled as 'On Loan'. For entries labeled as 'Free', we will leave the 'Beginning Year' and 'Ending Year' columns as null, indicating that the player does not have a contract and is available as a free agent.

In [15]:
# view rows with loan in contract col
fifa[fifa["Contract"].str.contains("Loan")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

In [16]:
# view rows with free in contract col
fifa[fifa["Contract"].str.contains("Free")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

Since the 'Beginning Year' column already includes the year portion from the left-hand side of the '~' when splitting the columns, we will use that year portion as the value for the 'Beginning Year' column.

In [17]:
fifa["Beginning Year"] = fifa["Beginning Year"].str.extract(r"(\d{4})")

In [18]:
# view rows with loan in contract col
fifa[fifa["Contract"].str.contains("Loan")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

In [19]:
# view rows with free in contract col
fifa[fifa["Contract"].str.contains("Free")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

For the 'Ending Year' column, we will extract the year portion from the 'Loan Date End' column and use it as the values for the 'Ending Year' column.

In [20]:
# fill null of ending year with the years of Loan Date End col
# place year of loan date end onto ending year
fifa.loc[fifa["Contract"].str.contains("Loan"), "Ending Year"] = fifa["Loan Date End"].dropna().str.extract(r"(\d{4})")[0]

In [21]:
# replace None with nan
fifa.loc[fifa["Contract"].str.contains("Free"), "Ending Year"] = np.nan

In [22]:
# view rows with loan in contract col
fifa[fifa["Contract"].str.contains("Loan")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

In [23]:
# view rows with free in contract col
fifa[fifa["Contract"].str.contains("Free")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

In [24]:
# check every row in contract col without ~ delimiter
fifa[~fifa["Contract"].str.contains("~")][["Contract","Loan Date End","Beginning Year","Ending Year"]]

In [25]:
fifa[~fifa["Contract"].str.contains("~")][["Contract","Loan Date End","Beginning Year","Ending Year"]].info()

After splitting the 'Contract' column, we will remove this column from our dataset since it is now unnecessary for analysis, as the information it contained has been transferred to the two newly created columns.

In [26]:
# all null values needed to be filled are filled, so we will drop the contract col
fifa.drop(columns="Contract", inplace=True)

In [27]:
fifa[["Loan Date End","Beginning Year","Ending Year"]] = fifa[["Loan Date End","Beginning Year","Ending Year"]].apply(pd.to_datetime)

### Height

In [28]:
# Convert Height Column to all units in cm and represent the datatype as a float

In [29]:
fifa["Height"].unique()

The 'Height' column currently contains a mix of imperial units (feet and inches) and SI units (centimeters). To standardize the units and ensure consistency, we will convert all height measurements to centimeters (cm).

In [30]:
fifa[~fifa["Height"].str.contains("cm")]["Height"].unique()

In [31]:
# Function to convert feet and inches to centimeters
def feet_inches_to_cm(height):
    if "'" in height and '"' in height:
        feet, inches = height.split("'")
        return int(feet) * 30.48 + int(inches.strip('"')) * 2.54  # 1 foot = 30.48 cm, 1 inch = 2.54 cm
    elif "'" in height:
        return int(height.replace("'", "")) * 30.48
    elif '"' in height:
        return int(height.replace('"', "")) * 2.54
    else:
        return int(height.split("cm")[0])

# Apply the function to the 'Height' column where it now has numbers representing only in cm
fifa['Height'] = fifa['Height'].apply(feet_inches_to_cm)

fifa["Height"]

In [32]:
fifa["Height"].unique()

In [33]:
fifa["Height"]

### Weight

In [34]:
# Convert Weight Column to all units in kg and represent the datatype as a float

In [35]:
fifa["Weight"].unique()

In [36]:
fifa[~fifa["Weight"].str.contains("kg")]["Weight"].unique()

Similar to the 'Height' column, the 'Weight' column also needs standardization to ensure measurements are in a single unit. Therefore, we will convert all weight values to kilograms (kg), which is the SI unit for weight.

In [37]:
# Function to convert pounds to kilograms
def lbs_to_kg(weight):
    if "lbs" in weight:
        return round(int(weight.replace("lbs", "")) * 0.45359237, 1) # 1 pound is approx 0.45359237 kg

    else:
        return int(weight.split("kg")[0])

# Apply the function to the 'Weight' column where it now has floats representing only in kg
fifa['Weight'] = fifa['Weight'].apply(lbs_to_kg)

In [38]:
# now weight col is in kg and all values are floats
fifa['Weight'].unique()

### Joined

Upon reviewing the 'Joined' column, we will transform this column so that its datatype is a datetime object, as all values represent dates.

In [39]:
# make the Joined Col into a datetime dtype
fifa["Joined"] = pd.to_datetime(fifa["Joined"])
fifa["Joined"]

In [40]:
# express value, wage, release clause cols in terms of thousands of euro

### Value, Wage, and Release Clause

The 'Value', 'Wage', and 'Release Clause' columns are currently not standardized and have different units. To ensure consistency, we will transform these columns to represent values in thousands of euros.

In [41]:
fifa[["Value", "Wage", "Release Clause"]]

In [42]:
fifa[~fifa["Value"].str.contains("K")]["Value"].unique()

In [43]:
fifa[~fifa["Wage"].str.contains("K")]["Wage"].unique()

In [44]:
fifa[~fifa["Release Clause"].str.contains("K")]["Release Clause"].unique()

In [45]:
fifa[~fifa["Value"].str.contains("K|M")]["Value"].unique()

In [46]:
fifa[~fifa["Wage"].str.contains("K|M")]["Wage"].unique()

In [47]:
fifa[~fifa["Release Clause"].str.contains("K|M")]["Release Clause"].unique()

In [48]:
# Function to convert Millions to thousands of euros
def euro_to_thousandEuros(money):
    amount = float(re.search(r'[\d.]+', money).group()) # extract numeric value
    if "M" in money:
        return amount * 1000
    elif "K" in money:
        return amount
    else:
        return amount / 1000

In [49]:
fifa[["Value", "Wage", "Release Clause"]] = fifa[["Value", "Wage", "Release Clause"]].apply(lambda x: x.apply(euro_to_thousandEuros))

### W/F. SM, and IR

The 'W/F', 'SM', and 'IR' columns contain a special star character, which is not ideal for making calculations. To address this, we will preserve only the numerical values in these columns.

In [50]:
# Extract the score from the W/F, SM, IR cols removing the star character
def number_extractor(str):
    return int(re.search(r'[\d.]+', str).group())

fifa[['W/F', 'SM', 'IR']] = fifa[['W/F', 'SM', 'IR']].apply(lambda x: x.apply(number_extractor))

In [51]:
fifa[fifa["Hits"].astype(str).str.contains("K")]["Hits"].unique()

In [52]:
fifa[~fifa["Hits"].astype(str).str.contains("K")]["Hits"].unique()

### Hits

Finally, we will transform the 'Hits' column since it contains a mix of string and integer values, where some values are represented with 'K' (indicating thousands) while others are represented as just floats or strings.

In [53]:
# Function to convert hits to number of hits as int64
def number_hits(hits):
    amount = float(re.search(r'[\d.]+', hits).group()) # extract numeric value
    if "K" in hits:
        return int(amount * 1000)
    else:
        return int(amount)

    
fifa["Hits"] = fifa["Hits"].astype(str).apply(number_hits)

In [54]:
fifa.info()

## Viewing the Final Cleaned Fifa Dataset

In [55]:
fifa

## Exporting File to CSV

At the conclusion of the data cleaning process, I will save the cleaned version of this dataset to a CSV file named 'cleaned_fifa_data.csv'. This file will serve as a reference for future analysis and visualizations.

In [56]:
# Export cleaned data to a CSV file
fifa.to_csv('cleaned_fifa_data.csv', index=False)